In [21]:
#Import tools/libraries

import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
import urllib
import io
import pandas as pd
import requests
from decorator import get_init
from numpy.lib.function_base import gradient

In [22]:
#Get the data
# Download the iris dataset from the internet
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
s = requests.get(url).content

col_names=["sepal_length", "sepal_width", "petal_length", "petal_width", "type"]
data = pd.read_csv(io.StringIO(s.decode('utf-8')), skiprows=1, header=None, names=col_names)
#data=pd.read_csv(pd.compat.StringIO(raw_data), skiprows=1, header=None, names=col_names)
data.head(10)

,sepal_length,sepal_width,petal_length,petal_width,type
0,4.9,3.0,1.4,0.2,Iris-setosa
1,4.7,3.2,1.3,0.2,Iris-setosa
2,4.6,3.1,1.5,0.2,Iris-setosa
3,5.0,3.6,1.4,0.2,Iris-setosa
4,5.4,3.9,1.7,0.4,Iris-setosa
5,4.6,3.4,1.4,0.3,Iris-setosa
6,5.0,3.4,1.5,0.2,Iris-setosa
7,4.4,2.9,1.4,0.2,Iris-setosa
8,4.9,3.1,1.5,0.1,Iris-setosa
9,5.4,3.7,1.5,0.2,Iris-setosa


In [25]:
#Node Class
class Node():
  def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
    """
    Constructor:
    We use object oriented programming because it is easier to assign atributes to each of the nodes
    Node is a class object tha always has these attributes.
    
    """

    #for decision node
    self.feature_index=feature_index #Integer representing the index of the feature used for splitting
    self.threshold=threshold #cutogg value for the feature used to split the data at that particular node in the tree
    self.left=left #Left child of this node
    self.right=right #Right child of this node
    self.info_gain=info_gain #Information gained by splitting this node

    #for leaf node
    self.value=value #Predicted value at this node

In [26]:
class DecisionTreeClassifier():
    def __init__(self, min_samples_split, max_depth):
        ''' 
        root: root node represents the entire dataset,
        and all subsequent nodes represent subsets of the data based on feature splits.

        min_samples_split: determines the minimum number of samples required to make a split at an internal node.
        If it has fewer than min_saples_split samples, the algorithm will not split that node any further and will become a leaf node.
        The appropriate value depends on the size and complexity of the dataset and should be chosen through cross-validation.

        max_depth: determines the depth of the decision tree. It is the length of the longest path from the root to the leaf node.
        It determines how many levels deep a decision tree can go

        Remember that a node has always 2 child nodes per level.
        '''
        
        # initialize the root of the tree 
        self.root = None #When the tree is initialized, there is no data to use for the root node
        
        # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth=0):
        '''
        Recursive function that implements the construction of the
        decision tree. It takes the input training data X and y, the
        current depth of the tree and maximum depth.

        At each call the function selects the best feature and corresponding
        split point that will produce the highest information gain using the
        find_best_split function. It splits the data based on the selected feature
        and split point creating a new node of the tree.

        If the maximum depth has not been reached and the current node has enough
        samples to split, the function recursively calls itself to build the left
        and right sub-trees. If either of these conditions is not met, the current
        node becomes a leaf node and its predicted class is set to the majority class
        of the samples in the node.

        It then returns the root node of the decision tree, which can be used to make
        predictions on new data.
        ''' 
        
        X, Y = dataset[:,:-1], dataset[:,-1]
        #[:,:-1] select all rows and columns except the last column (features)
        #[:,-1] select all rows and only the last column (labels/target variables we want to predict)
        num_samples, num_features = np.shape(X)
        
        # split until stopping conditions are met
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # check if information gain is positive
            if best_split["info_gain"]>0:
                # recur left
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                # recur right
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                # return new decision node
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["info_gain"])
                # Returns node with the index of the feature that produced the best split in the current node
                # Its threshold, the left/right subtree, the information gain resulting from the split
        
        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        '''
        Finds the best split for a given dataset. Takes as input the dataset, number of samples, and features.
        Creates a dictionary to store the details of the best split.

        The function loops over all features in the dataset and gets all the unique values of the current feature.
        For each unique value, it splits the dataset into left/right child nodes based on whether the feature
        is less than or greater than the threshold.

        For each split, the function computes the information gain using the Gini index and updates the best_split
        dictionary if the current information gain is greater thn the previous maximum information gain.

        Finally, the function returns the best_split dictionary, which contains the parameters for the best_split.
        '''
        
        # dictionary to store the best split
        best_split = {}
        max_info_gain = -float("inf")
        
        # loop over all the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index] # Grab the values of that specific feature_index
            possible_thresholds = np.unique(feature_values) # Set as possible thresholds to study the unique values of this new dataset "feature_values"
            # loop over all the feature values present in the data
            for threshold in possible_thresholds:
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if childs are not null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # compute information gain
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    # if the current info_gain is more than the previous one, update parameters of dictionary:
                    if curr_info_gain>max_info_gain:
                        
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
                        
        # return best split
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        '''
        Method that grabs the original dataset, feature index and threshold.
        It returns a tuple (dataset_left, dataset_right) which are two new numpy arrays
        containing:

        dataset_left (right): An array that contains all the rows from dataset where the value
        of the feature at feature_index is <= (>) to threshold.
        '''
        
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def information_gain(self, parent, l_child, r_child, mode="entropy"):
        '''
        Computes information gain for a given split.

        parent: Target variable of the parent node.
        l/r_child: Target variables of the left/right child nodes from split.
        mode: Mode to compute the entropy
        '''
        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        # Counts the proportion of points that belong to the left (right) child with respect the previous parent node


        if mode=="gini":
            gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))
            # Uses gini mode to compute information gain
        else:
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
            # Uses default mode to compute information gain
        return gain
    
    def entropy(self, y):
        '''
        Method that computes entropy of a set of labels y.
        Entropy is a measure of impurity of a set. The entropy is at its minimum (0) when all labels in the set belong
        to the same class (the set is pure), and it is at its maximum (1) when the set is equally divided among all possible labels.
        
        Finally, it returns the entropy.
        '''
        
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            # len(y[y==cls]) prompts the number of rows that are of type "cls"
            entropy += -p_cls * np.log2(p_cls)
        return entropy
    
    def gini_index(self, y):
        '''
        Calculates the gini index of a set of class labels. It is a measure of impurity of a set of labels.
        0 means completely pure (all labels in the set are the same) and 1 completely impure (labels in the set are evenly distributed)
        
        Returns the gini_index.
        '''
        
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y) #prob. of that class
            gini += p_cls**2
        return 1 - gini
        
    def calculate_leaf_value(self, Y):
        '''
        Returns the label that appears the most frequently in the given set of labels (Y) of that node.
        This most frequent label will be assigned to the leaf node.
        '''
        
        Y = list(Y)
        return max(Y, key=Y.count)
    
    def print_tree(self, tree=None, indent=" "):
        '''
        Prints the decision tree structure in a readable format.
        '''
        
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def fit(self, X, Y):
        '''
        Trains the decision tree on a given concatenated dataset X+Y. Builds a tree using this dataset
        finding the best split at each node and uses the root attribute to set it to the root node of
        the constructed decision tree. Then, this decision tree is used for prediction on new data.

        Assigning the root attribute of the DecisionTreeClassifier() object to the root node of the
        decision tree means that the object now has access to the entire decision tree structure and
        can use it to make predictions on new data.

        The root node contains all the information needed to classify new samples by traversing the
        tree and making decisions at each internal node based on the feature values of the new sample.

        The best parameters that were learned during training, such as optimal feature to split on at
        each internal node and the threshold for that feature, are encoded in the decision tree structure.
        The root node represents the first decision point in the tree, and subsequent decisions are made
        by following the appropriate branches until a leaf node is reached, which corresponds to the final
        classification for the input sample.
        '''
        
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
    
    def predict(self, X):
        '''
        Takes dataset X and predicts the class labels for each sample in the dataset using the decision tree
        that has been trained earlier.

        It returns a list of predicted labels.
        '''
        
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions
    
    def make_prediction(self, x, tree):
        '''
        Predicts the label of a single data point by traversing down the decision tree until a leaf node is reached.
        The prediction is made based on the majority class of the training examples that reach that leaf node.
        '''
        
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

In [27]:
#Train/Test split

X=data.iloc[:,:-1].values
Y=data.iloc[:,-1].values.reshape(-1,1)
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2, random_state=41)

In [34]:
#Fit the model

classifier=DecisionTreeClassifier(min_samples_split=2, max_depth=30)
classifier.fit(X_train,Y_train)
classifier.print_tree()

X_2 <= 1.7 ? 0.33904421497105636
 left:Iris-setosa
 right:X_3 <= 1.5 ? 0.40269559500328744
  left:X_2 <= 4.9 ? 0.04996712689020377
    left:Iris-versicolor
    right:Iris-virginica
  right:X_2 <= 4.8 ? 0.040912933220625364
    left:X_1 <= 2.8 ? 0.5
        left:Iris-virginica
        right:Iris-versicolor
    right:X_3 <= 1.7 ? 0.026938775510204082
        left:X_0 <= 6.7 ? 0.5
                left:Iris-versicolor
                right:Iris-virginica
        right:Iris-virginica


In [35]:
#Test the model

Y_pred=classifier.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_pred)

0.9